<a href="https://colab.research.google.com/github/OrensteinLab/PrimerDesigner/blob/main/Tracking_Timings_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pre-Calculations

##Installs & Imports

In [ ]:
!pip install -U bokeh seaborn pandas
!pip install git+https://github.com/FordyceLab/seequence.git#egg=seequence
!pip install primer3-py biopython pandarallel
!pip install pulp
!pip install gurobipy

  Cloning https://github.com/FordyceLab/seequence.git to /tmp/pip-install-8itr2qw6/seequence_3fc93bd6d0ce471088d6de980f896256
  Running command git clone --filter=blob:none --quiet https://github.com/FordyceLab/seequence.git /tmp/pip-install-8itr2qw6/seequence_3fc93bd6d0ce471088d6de980f896256
  Resolved https://github.com/FordyceLab/seequence.git to commit 3ea730537fcf5b7ef807ebf6e057f5bf4e875bb9
  Preparing metadata (setup.py) ... done


In [ ]:
# IMPORTS
import time

import random as rand

from seequence.view import *
from seequence.color import *

from pandarallel import pandarallel as pl
pl.initialize()

import primer3 as p3
from Bio.Seq import Seq
from Bio.SeqUtils import GC, seq1, seq3
from Bio.SeqUtils.CodonUsage import SharpEcoliIndex, SynonymousCodons

import itertools as it
import numpy as np
import pandas as pd
pd.set_option('display.precision', 1)
import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.models.annotations import Span
from bokeh.models import Select
from bokeh.layouts import column
from IPython.display import display, clear_output

import gurobipy as gp

import pickle

import tracemalloc
import matplotlib.pyplot as plt

def revcomp(seq):
  return str(Seq(seq).reverse_complement())
def translate(seq):
  return str(Seq(seq).translate())

# clear_output()
print('Ready')

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/usr/local/lib/python3.10/dist-packages/Bio/SeqUtils/CodonUsageIndices.py:16: BiopythonDeprecationWarning: This module has been DEPRECATED. Please use the CodonAdaptationIndex class in Bio.SeqUtils instead. Note that this class has been updated to use modern Python, and may give slightly different results from the CodonAdaptationIndex class in Bio.SeqUtils.CodonUsage, as the code was changed to be consistent with the published paper by Sharp and Li. The code in the old CodonAdaptationIndex class in Bio.SeqUtils.CodonUsage was not changed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/SeqUtils/CodonUsage.py:15: BiopythonDeprecationWarning: This module has been DEPRECATED. Please use the CodonAdaptationIndex class in Bio.SeqUtils instead. Note that this class has been updated to use modern Python, and may give slightly different results from the CodonAdaptationIndex class in Bio.SeqUtils.CodonUsage, as the code was changed to be consistent with the published paper by Sharp

Ready


##Codon Table + Important Functions

In [ ]:
lst = []
for k,v in SynonymousCodons.items():
  if k!='STOP':
    for vv in v:
      lst.append((seq1(k), k, vv, SharpEcoliIndex[vv]))
codon_df = pd.DataFrame(lst, columns=['c1','c3','codon','freq']).sort_values(['c1','freq'], ascending=[True,False])
codon_df

,c1,c3,codon,freq
30,A,ALA,GCT,1.0e+00
27,A,ALA,GCA,5.9e-01
29,A,ALA,GCG,4.2e-01
28,A,ALA,GCC,1.2e-01
1,C,CYS,TGC,1.0e+00
...,...,...,...,...
55,V,VAL,GTG,2.2e-01
54,V,VAL,GTC,6.6e-02
52,W,TRP,TGG,1.0e+00
60,Y,TYR,TAC,1.0e+00


In [ ]:
def create_mut_df():
  mut_df = pd.DataFrame(columns=['lib_i','wt_aa','pos_aa','mut_aa']) #library #, wildtype, position, mutation
  mut_df.pos_aa = mut_df.pos_aa.astype(int)

  # GENERATE (V scan):
  scan = pd.DataFrame()
  scan['wt_aa'] = list(mutreg_aa)
  scan['pos_aa'] = list(range(len(mutreg_aa)))
  scan['mut_aa'] = ['V' if aa!='V' else 'A' for aa in mutreg_aa] #V: A, Else: V
  scan['lib_i'] = 0
  mut_df = pd.concat([mut_df, scan])

  # GENERATE (P scan):
  scan = pd.DataFrame()
  scan['wt_aa'] = list(mutreg_aa)
  scan['pos_aa'] = list(range(len(mutreg_aa)))
  scan['mut_aa'] = ['P' if aa!='P' else 'A' for aa in mutreg_aa]#P: A, Else: P
  scan['lib_i'] = 0
  #mut_df = mut_df.append(scan)
  mut_df = pd.concat([mut_df, scan])

  # GENERATE (G scan):
  scan = pd.DataFrame()
  scan['wt_aa'] = list(mutreg_aa)
  scan['pos_aa'] = list(range(len(mutreg_aa)))
  scan['mut_aa'] = ['G' if aa!='G' else 'A' for aa in mutreg_aa]#G: A, Else: G
  scan['lib_i'] = 1
  #mut_df = mut_df.append(scan)
  mut_df = pd.concat([mut_df, scan])

  # v DO NOT EDIT v --------------------------------------------------------------
  # nt (codon) info
  mut_df['start_nt'] = 3*mut_df.pos_aa
  mut_df['stop_nt'] = mut_df.start_nt+3
  mut_df['wt_nt'] = mut_df.apply(lambda row: mutreg_nt[row.start_nt:row.stop_nt], axis=1) #finds codon between start & stop for a given row
  mut_df['mut_nt'] = codon_df.set_index('c1').query('freq==1.0').loc[mut_df.mut_aa,'codon'].values #finds the codon (with freq 1) that produces the new mutated aa

  mut_df = mut_df.sort_values(['lib_i','pos_aa','mut_aa']).reset_index(drop=True)
  return mut_df

In [ ]:
def calcOffTarget(primer, seq, start):
  fl,fr = seq[:primer.start-start], seq[primer.stop-start:]
  rl,rr = revcomp(fl), revcomp(fr)

  res_fl = pcr.calcHeterodimer(primer.seq, fl).todict()
  res_fr = pcr.calcHeterodimer(primer.seq, fr).todict()
  res_rl = pcr.calcHeterodimer(primer.seq, rl).todict()
  res_rr = pcr.calcHeterodimer(primer.seq, rr).todict()

  ot_tm = max(res_fl['tm'], res_fr['tm'], res_rl['tm'], res_rr['tm'])
  # ot_dg = min(res_fl['dg'], res_fr['dg'], res_rl['dg'], res_rr['dg'])*1e-3
  return ot_tm

def n_subsequences(sequence, lmin, lmax):
  print(sum(len(sequence) - l + 1 for l in range(lmin, lmax+1)))

def subsequences(sequence, lmin, lmax): #Generates all subsequences w/ all poss. start-stop pairs
  ls = []
  for j in range(lmin, lmax+1): #length
    for i in range(len(sequence)-j+1): #starting index
      start = i
      stop = i+j
      ls.append([sequence[start:stop], start, stop, stop-start])
  return pd.DataFrame(ls, columns=['seq','start','stop','len'])

In [ ]:
def create_primer_df():
  # convention: start index of r-primers will be 3' (i.e. start < stop)
  primer_f = pd.DataFrame(columns=['seq','start','stop','fr','len'])
  primer_f[['seq','start','stop','len']] = subsequences(sequence_nt, primer_lmin, primer_lmax)
  primer_f['fr'] = 'f'

  #Shifting so that 0 is at the start of mutreg (upstream has negative values)
  primer_f['start'] = primer_f.start - mutreg_start
  primer_f['stop'] = primer_f.stop - mutreg_start

  #Creating reverse primers at same locations
  primer_r = primer_f[['seq','start','stop','fr','len']].copy()
  primer_r['fr'] = 'r'
  primer_r['seq'] = primer_r.seq.apply(revcomp)

  #Concatenating Forward & Reverse
  primer_df = pd.concat([primer_f,primer_r])
  primer_df.sort_values(by=['start','stop','fr'], inplace=True)

  #Calculating "Cost" Values
  primer_df['gc'] = primer_df.seq.apply(GC)
  primer_df['tm'] = primer_df.seq.apply(pcr.calcTm)
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHairpin(s).todict())
  primer_df['hp_tm'] = res.apply(lambda res: res['tm'])
  primer_df['hp_dg'] = res.apply(lambda res: res['dg']*1e-3)
  res = primer_df.seq.parallel_apply(lambda s: pcr.calcHomodimer(s).todict())
  primer_df['ho_tm'] = res.apply(lambda res: res['tm'])
  primer_df['ho_dg'] = res.apply(lambda res: res['dg']*1e-3)

  def primer_cost(primer):
    tm_min = 58
    hp_dg_max = -5
    ho_dg_max = -5

    tm_cost = max(0, tm_min-primer.tm)**1.5
    gc_cost = 0
    hp_cost = max(0, hp_dg_max - primer.hp_dg)**1.2
    ho_cost = max(0, ho_dg_max - primer.ho_dg)**1.2
    len_cost = primer.len*1e-5  # tiebreaker (nudge towards shorter primers)

    cost = tm_cost + gc_cost + hp_cost + ho_cost + len_cost
    return cost

  primer_df['cost'] = primer_df.parallel_apply(primer_cost, axis=1)
  primer_df['log10cost'] = primer_df.cost.apply(np.log10)

  primer_df.reset_index(inplace=True)
  primer_f = primer_df.query('fr=="f"').reset_index(drop=True)
  primer_r = primer_df.query('fr=="r"').reset_index(drop=True)
  primer_df.set_index(['start','stop','fr'], inplace=True)

  return primer_f, primer_df

In [ ]:
class Primer:
  def __init__(self,start,stop,is_r=False):
    assert start < stop
    self.start = start
    self.stop = stop
    self.is_r = is_r #forward or reverse
    self.l = stop-start #length
    self.w = primer_df.at[self.tup(),'cost'] #total cost value; the fancy notation is b/c
                                             #of the hierarchal lookup system in panda.df

  def __str__(self):
    return ' '.join(map(str,(self.start, self.stop, self.is_r)))
  def __repr__(self):
    return f'{("r" if self.is_r else"f")}({self.start},{self.stop})'
  def tup(self):
    return (self.start,self.stop,("r" if self.is_r else"f"))


def actions(primer): #returning possible counterparts (forward -> reverse; reverse -> forward)
                     #i.e. this method gets the "neighbors"
  if not primer.is_r:  # fwd
    for oligo_l, primer_l in it.product(reversed(range(oligo_lmin, oligo_lmax+1)),
                                        range(primer_lmin,primer_lmax+1)):

      stop = primer.start + oligo_l
      start = stop - primer_l
      yield Primer(start, stop, is_r=True)

  elif primer.is_r:  # rev
    for overlap_l, primer_l in it.product(reversed(range(overlap_lmin, overlap_lmax+1)),
                                          range(primer_lmin,primer_lmax+1)):

      start = primer.stop - overlap_l
      stop = start + primer_l

      # filter
      no_split = (primer.start - stop) >= primer.start%3
      if (stop > primer.start) or (not no_split): ## redundant to check first condition?
        continue
      yield Primer(start, stop)

In [ ]:
def dfs(primer): #CREATING the graph
  if (primer.start >= mutreg_l) and primer.is_r:  # base case (end)
    G.add_edge(primer.tup(),'d', weight=0.) #G is global variable defined in next section
    return

  for next_primer in actions(primer):
    is_new = not G.has_node(next_primer.tup())
    G.add_edge(primer.tup(),next_primer.tup(), weight=next_primer.w) #weight is the cost of the new primer
    if is_new:
      dfs(next_primer)

def paths_ct(G, u, d): #total # of paths between two points
    if u == d:
        return 1
    else:
        if not G.nodes[u]: #npaths attribute is the # of paths out of u
            G.nodes[u]['npaths'] = sum(paths_ct(G, c, d) for c in G.successors(u))
        return G.nodes[u]['npaths']

In [ ]:
def run_greedy(G, primer_f, primer_df):
  path_ls = [[]]
  G_sub = G.copy()
  for i in range(num_proteins):
    nodes_to_remove = []
    for p,n in it.product(path_ls[-1],G_sub.nodes()):
      if n=='s' or n=='d':
        continue
      pn_intersect = n[1]-p[0] > allowed_overlap and p[1]-n[0] > allowed_overlap and n[2]==p[2]  ## check overlap
      if pn_intersect:
        nodes_to_remove.append(n)
    G_sub.remove_nodes_from(set(nodes_to_remove))
    try:
      path_ls.append([primer for primer in nx.algorithms.shortest_path(G_sub,'s','d', weight='weight')][1:-1])
    except:
      print(f'WARNING: No feasible primer sequence for lib_{i}; reduce number of libraries or relax constraints.')

  path_ls = path_ls[1:]

  # # https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
  primer_set = pd.DataFrame()
  for i,primer_ls in enumerate(path_ls):
    primer_set1 = primer_df.loc[primer_ls].copy().reset_index()
    primer_set1['lib_i'] = i
    primer_set = pd.concat([primer_set, primer_set1])
  primer_set['tile_i'] = primer_set.groupby(['lib_i','fr']).cumcount()
  primer_set = primer_set[['lib_i','tile_i']+primer_set.columns[:-2].to_list()]

  #print(primer_set.groupby('lib_i').cost.sum())
  cost = primer_set.cost.sum()

  return path_ls, cost

##ILP Functions

In [ ]:
def get_model():
  params = {
  "WLSACCESSID": "7d564052-fa88-4170-8367-f26ba5820051",
  "WLSSECRET": "a37ba3fa-edcd-4bdf-987d-1ba70d1086f1",
  "LICENSEID": 2423852,
  }
  env = gp.Env(params=params)

  # Create the model within the Gurobi environment
  model = gp.Model('min-sum', env=env)
  return model

In [ ]:
def get_info(graphs):
  setup_start = time.time()
  tracemalloc.start()
  #Create Model
  model = get_model()
  #Getting Nodes/Edges
  graph_edges = G.edges(data=True)
  graph_nodes = [node for node in G.nodes if node != 's' and node != 'd'] #removing s & d nodes

  #for G in graphs:
   # graphs_edges.append(G.edges(data=True))
    #graphs_nodes.append(sorted((node for node in G.nodes if node != 's' and node != 'd'), key = lambda x:x[0:2])) #removing s & d

  def create_bins(): #bins take the form (start, end, [])
    all_bins = {(start, start+len, c):[] for start in range(*nt_range) for len in range(*l_range) for c in ('f', 'r')}

    for node in graph_nodes: #for each node, add it into all the necessary bins
      for bin_start in range(node[0], node[1]):
        for bin_length in range(*l_range):
          if bin_start + bin_length > node[1]:
            break
          else:
            all_bins[(bin_start, bin_start + bin_length, node[2])].append(node)

    all_bins = {key:val for key,val in all_bins.items() if val} #no empty bins
    return all_bins

  all_bins = create_bins()
  print("Number of Constraints:", len(all_bins))
  print("Average Vars Per Constraint", 1/len(all_bins) * sum(len(val) for _, val in all_bins.items()))

  #Converting Graphs to Lists
  ij = gp.tuplelist()
  w_ij = gp.tupledict()

  for edge in graph_edges:
    l = (str(edge[0]), str(edge[1])) #i, j
    ij.append(l)
    w_ij[l] = edge[-1]['weight']

  print("Finished Conversion")

  #Adding Variables
  x = model.addVars(ij, obj=w_ij, vtype=gp.GRB.BINARY)
  print("Finished Variable Creations")

  #Intersection Constraints
  for cnt, nodes in enumerate(all_bins.values()):
    all_edges = []
    if cnt % (len(all_bins)//25) == 0:
          print(int(cnt / len(all_bins) * 100))
    for node in nodes:
      all_edges.append(x.sum(str(node), '*'))
    model.addConstr(gp.quicksum(all_edges) <= 1)
  print("Finished Intersection constraints!")

  #Single Path Constraints
  for n in graph_nodes + ['s', 'd']: #adding s & d back just here
    v = str(n)
    model.addConstr(sum(x[i,j] for i,j in ij.select(v, '*')) - sum(x[j,i] for j,i in ij.select('*', v)) == (num_proteins if v=='s' else -1 * num_proteins if v=='d' else 0), v)

  setup_time = time.time() - setup_start
  setup_memory = tracemalloc.get_traced_memory()[1] / 10**6
  tracemalloc.stop()

  tracemalloc.start()
  start_time = time.time()
  model.optimize()
  ILP_time = time.time() - start_time
  ILP_memory = tracemalloc.get_traced_memory()[1] / 10**6
  tracemalloc.stop()

  def post_processing(variables):
    all_proteins = [['s'] for _ in range(num_proteins)]
    true_edges = [index for index, var in x.items() if var.X != 0]

    while true_edges:
      edge = true_edges.pop(0)
      added_edge = False
      for protein_list in all_proteins:
        if edge[0] == protein_list[-1]:
          protein_list.append(edge[1])
          added_edge = True
          break
      if not added_edge:
        true_edges.append(edge)

    return all_proteins

  actual_values = post_processing(x)
  for cnt, vals in enumerate(actual_values):
    print(f"Protein #{cnt+1} ({len(vals)})")
    print(vals)
    print()

  return model.numVars, model.numConstrs, setup_time, setup_memory, ILP_time, ILP_memory, actual_values, model.objVal

##Full Sequence

In [ ]:
upstream_nt = 'ATTTGAATGTATTTAGAAAAATAAACAAATAGGGGTTCCGCGCACATTTCCCCGAAAAGTGCTAGTGGTGCTAGCCCCGCGAAATTAATACGACTCACTATAGGGTCTAGAAATAATTTTGTTTAACTTTAAGAAGGAGATATACATATG'
mutreg_nt_full = 'CAAAGCCCAGCACCTGCCGCAGCGCCTGCCCCTGCGGCACGTTCCATCGCAGCTACGCCTCCTAAACTGATCGTGGCAATTAGCGTGGACCAGTTTAGTGCAGACTTGTTCTCGGAGTATCGTCAATATTACACCGGAGGTTTAAAGCGTCTTACATCCGAAGGAGCTGTGTTCCCACGTGGTTATCAGAGTCATGCGGCAACAGAAACGTGTCCTGGTCACTCAACGATCCTGACAGGATCACGTCCGTCACGTACGGGTATTATCGCTAATAACTGGTTCGACTTGGACGCAAAGCGTGAGGATAAAAATCTGTACTGTGCTGAGGATGAATCCCAACCCGGTAGTTCGTCTGACAAGTACGAAGCTTCGCCACTGCACTTAAAGGTACCCACCCTGGGGGGACGCATGAAAGCCGCCAATCCTGCGACTCGTGTCGTCTCTGTTGCCGGCAAGGATCGCGCGGCCATTATGATGGGTGGCGCCACAGCGGATCAGGTCTGGTGGTTAGGGGGGCCTCAGGGGTATGTTTCGTATAAGGGTGTAGCGCCAACTCCCCTTGTAACACAGGTCAATCAGGCCTTTGCACAGCGCTTAGCTCAGCCGAACCCGGGATTTGAGTTGCCTGCTCAGTGCGTCAGCAAGGACTTTCCTGTTCAAGCGGGAAATCGCACAGTGGGTACCGGCCGCTTCGCCCGTGATGCTGGTGACTACAAAGGTTTTCGCATTTCCCCGGAGCAGGATGCTATGACGCTTGCATTCGCTGCCGCGGCCATTGAAAATATGCAATTAGGGAAGCAGGCCCAGACCGATATTATTAGCATTGGACTGAGCGCTACGGATTACGTGGGACACACCTTCGGCACGGAGGGTACGGAGAGTTGCATCCAAGTGGATCGTTTAGACACGGAGCTTGGTGCATTCTTTGATAAACTGGATAAGGATGGGATTGACTACGTAGTAGTGCTGACTGCAGATCATGGAGGACACGATCTGCCCGAACGTCATCGTATGAATGCCATGCCGATGGAACAGCGCGTAGACATGGCCCTGACACCTAAAGCTCTGAATGCTACCATCGCTGAGAAAGCTGGCCTTCCGGGCAAAAAGGTTATTTGGTCAGATGGACCTTCTGGCGATATTTACTATGATAAGGGCCTTACAGCCGCTCAACGTGCCCGTGTTGAAACCGAGGCGTTAAAATACTTGCGCGCGCATCCCCAAGTACAGACTGTATTCACTAAGGCGGAAATCGCGGCTACCCCTTCTCCGTCGGGACCACCTGAGAGCTGGAGTTTGATCCAGGAAGCTCGCGCGTCATTTTACCCGTCGCGCTCCGGGGACCTGTTACTTTTATTGAAACCTCGTGTGATGAGCATTCCTGAGCAAGCAGTCATGGGCTCGGTTGCAACCCATGGATCTCCATGGGATACGGATCGCCGTGTGCCTATCCTGTTTTGGCGCAAAGGTATGCAGCATTTCGAACAACCCTTAGGAGTAGAGACTGTTGATATTTTGCCCTCCTTGGCTGCACTTATTAAGCTTCCTGTTCCTAAGGATCAGATCGACGGCCGCTGTCTGGACTTGGTCGCCGGCAAGGATGATTCCTGTGCTGGACAG'
downstream_nt = 'GGAGGAGGGTCTGGGGGAGGAGGCAGTGGCATGGTGAGCAAGGGCGAGGAGCTGTTCACCGGGGTGGTGCCCATCCTGGTCGAGCTGGACGGCGACGTAAACGGCCACAAGTTCAGCGTGTCCGGCGAGGGCGAGGGCGATGCCACCTACGGCAAGCTGACCCTGAAGTTCATCTGCACCACCGGCAAGCTGCCCGTGCCCTGGCCCACC'

#Per-Paramter Code

##Creating Graphs

##Parameters

In [ ]:
all_data = []
seq_length = 102 #102 - 150 [+= 3]
primer_lmin, primer_lmax = 18, 30 # PARAM: primer lengths (inclusive)
pcr = p3.thermoanalysis.ThermoAnalysis(dna_conc= 250,
                                       mv_conc= 50,
                                       dv_conc= 0,
                                       dntp_conc= 0,
                                       tm_method= 'santalucia',
                                       salt_correction_method= 'owczarzy',
                                       temp_c= 25)
overlap_lmin,overlap_lmax = 45,50
oligo_lmin,oligo_lmax = 195,205
num_proteins = 3
allowed_overlap = 6

##Code To Run

In [ ]:
for seq_length in range(302, 353, 3):
  mutreg_nt = mutreg_nt_full[:seq_length]
  sequence_nt = upstream_nt + mutreg_nt + downstream_nt
  mutreg_l = len(mutreg_nt)
  mutreg_start = len(upstream_nt)
  mutreg_stop = mutreg_start + mutreg_l
  mutreg_aa = translate(mutreg_nt)

  nt_range = (-len(upstream_nt), len(mutreg_nt) + len(downstream_nt)+1) #range of nucleotides
  l_range = (allowed_overlap+1, primer_lmax+1)

  mut_df = create_mut_df()
  primer_f, primer_df = create_primer_df()

  #Creating the Graph
  start_time = time.time()
  tracemalloc.start()
  G = nx.DiGraph()
  primers_init = [Primer(p.start,p.stop) for _,p in primer_f.query('stop<=0')[['start','stop']].iterrows()]  ## all forward primers that end before mutreg
  for primer in primers_init:
    G.add_edge('s',primer.tup(), weight=primer.w) #intializing the s-primer connection
    dfs(primer) #create the rest of the graph

  graph_time = int(time.time() - start_time)
  graph_memory = tracemalloc.get_traced_memory()[1] / 10**6 #MB
  tracemalloc.stop()

  #Greedy Solution
  start_time = time.time()
  greedy_solution, greedy_obj = run_greedy(G, primer_f, primer_df)
  greedy_time = time.time() - start_time

  #Convert to ILP & Solve
  tracemalloc.start()
  start_time = time.time()

  numVars, numConstrs, setup_time, setup_memory, ILP_time, ILP_memory, actual_values, objective = get_info([G, G])
  #numVars, numConstrs, setup_time, setup_memory, ILP_time, ILP_memory = 0,0,0,0,0,0
  all_data.append({"seq_length": seq_length,
                    "Nodes":len(G.nodes),
                    "Edges": len(G.edges),
                    "Time (Graph)": graph_time,
                    "MP (Graph)": graph_memory,
                    "Vars": numVars,
                    "Constr": numConstrs,
                    "Time (Setup)": setup_time,
                    "MP (Setup)": setup_memory,
                    "Time (ILP)": ILP_time,
                    "MP (ILP)": ILP_memory,
                    "ILP Solution": actual_values,
                    "ILP Objective": objective,
                    "Greedy Solution": greedy_solution,
                    "Greedy Objective": greedy_obj,
                    "Greedy Time": greedy_time})
  print(all_data[-1])

In [ ]:
all_data = all_data[1:]

In [ ]:
print(all_data[0].keys())
with open('data.csv', 'w') as f:
  w = csv.DictWriter(f,all_data[-1].keys())
  w.writeheader()
  w.writerows(all_data)




dict_keys(['seq_length', 'Nodes', 'Edges', 'Time (Graph)', 'MP (Graph)', 'Vars', 'Constr', 'Time (Setup)', 'MP (Setup)', 'Time (ILP)', 'MP (ILP)', 'ILP Solution', 'ILP Objective', 'Greedy Solution', 'Greedy Objective'])
